**Library Imports**

In [1]:
import pandas as pd
import io
import requests
import numpy as np
import os
import seaborn as sns

from scipy.stats import zscore

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, load_model

from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from IPython.display import display

from imblearn.over_sampling import SMOTE

import tensorflow as tf

Panda Preferences

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

Read CSV

In [3]:
df_train = pd.read_csv("../../../../../Datasets/Flows/train.csv")
display(df_train)

,service,orig_bytes,resp_bytes,orig_pkts,resp_pkts,flow_duration,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,bwd_header_size_tot,fwd_pkts_payload.tot,fwd_pkts_payload.avg,bwd_pkts_payload.tot,bwd_pkts_payload.avg,flow_pkts_payload.tot,flow_pkts_payload.avg,fwd_iat.tot,fwd_iat.avg,bwd_iat.tot,bwd_iat.avg,flow_iat.tot,flow_iat.avg,active.tot,active.avg,traffic,proto_1,proto_2,proto_3,is_attack
0,0,-0.004520,-0.002387,-0.084516,-0.431610,-0.292571,0.197830,-0.289197,-0.118756,-0.119291,-0.119111,1.148129,0.204570,0.274130,0.277337,1.153704,-0.277044,-0.293967,0.033529,0.472366,-0.304146,-0.299700,-0.293435,-0.294249,-0.306244,-0.296590,-0.306244,-0.306244,slowloris,1,0,0,1
1,0,-0.004520,-0.002387,-0.124520,-0.431610,-0.292571,-0.454575,-0.289197,-0.118756,-0.119291,-0.119111,-0.753681,-0.481517,-0.478594,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306564,-0.299233,-0.306564,-0.306564,camoverflow,1,0,0,1
2,0,-0.004520,-0.002387,-0.124520,-0.431610,-0.292571,-0.454575,-0.289197,-0.118756,-0.119291,-0.119111,-0.753681,-0.481517,-0.478594,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306564,-0.299233,-0.306564,-0.306564,camoverflow,1,0,0,1
3,0,-0.004520,-0.002387,-0.084516,2.035536,-0.292547,-0.454575,-0.289197,1.283487,1.288746,1.287058,1.148129,0.033048,-0.008142,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306540,-0.299035,-0.306540,-0.306540,netscan,1,0,0,1
4,0,-0.004520,-0.002387,-0.124520,-0.431610,-0.292571,-0.454575,-0.289197,-0.118756,-0.119291,-0.119111,-0.753681,-0.481517,-0.478594,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306564,-0.299233,-0.306564,-0.306564,camoverflow,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052460,0,-0.004520,-0.002387,-0.124520,-0.431610,-0.292571,-0.454575,-0.289197,-0.118756,-0.119291,-0.119111,-0.753681,-0.481517,-0.478594,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306564,-0.299233,-0.306564,-0.306564,camoverflow,1,0,0,1
2052461,0,-0.004520,-0.002387,-0.124520,-0.431610,-0.292571,-0.454575,-0.289197,-0.118756,-0.119291,-0.119111,-0.753681,-0.481517,-0.478594,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306564,-0.299233,-0.306564,-0.306564,camoverflow,1,0,0,1
2052462,0,-0.004520,-0.002387,-0.124520,-0.431610,-0.292571,-0.454575,-0.289197,-0.118756,-0.119291,-0.119111,-0.753681,-0.481517,-0.478594,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306564,-0.299233,-0.306564,-0.306564,camoverflow,1,0,0,1
2052463,0,-0.004520,-0.002387,-0.124520,-0.431610,-0.292571,-0.454575,-0.289197,-0.118756,-0.119291,-0.119111,-0.753681,-0.481517,-0.478594,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306564,-0.299233,-0.306564,-0.306564,camoverflow,1,0,0,1


In [4]:
df_test = pd.read_csv("../../../../../Datasets/Flows/test.csv")
display(df_test)

,service,orig_bytes,resp_bytes,orig_pkts,resp_pkts,flow_duration,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,bwd_header_size_tot,fwd_pkts_payload.tot,fwd_pkts_payload.avg,bwd_pkts_payload.tot,bwd_pkts_payload.avg,flow_pkts_payload.tot,flow_pkts_payload.avg,fwd_iat.tot,fwd_iat.avg,bwd_iat.tot,bwd_iat.avg,flow_iat.tot,flow_iat.avg,active.tot,active.avg,traffic,proto_1,proto_2,proto_3,is_attack
0,0,-0.004520,-0.002387,-0.124520,-0.431610,-0.292571,-0.454575,-0.289197,-0.118756,-0.119291,-0.119111,-0.753681,-0.481517,-0.478594,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306564,-0.299233,-0.306564,-0.306564,camoverflow,1,0,0,1
1,0,-0.004520,-0.002387,-0.084516,2.035536,-0.292381,-0.454575,-0.289197,0.060849,0.061057,0.060998,1.148129,0.033048,-0.008142,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306377,-0.297687,-0.306377,-0.306377,netscan,1,0,0,1
2,0,-0.004520,-0.002387,-0.124520,-0.431610,-0.292571,-0.454575,-0.289197,-0.118756,-0.119291,-0.119111,-0.753681,-0.481517,-0.478594,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306564,-0.299233,-0.306564,-0.306564,camoverflow,1,0,0,1
3,2,-0.004517,-0.002387,0.075498,-0.431610,3.522102,2.807449,-0.289197,-0.118712,-0.119246,-0.119066,1.148129,2.948919,3.285027,0.322464,-0.005740,-0.277044,-0.293967,0.059874,-0.133772,3.434651,3.368824,3.521460,3.387436,3.431807,3.134471,3.431807,3.431807,normal,1,0,0,0
4,0,-0.004520,-0.002387,-0.084516,2.035536,-0.292487,-0.454575,-0.289197,0.285594,0.286730,0.286372,1.148129,0.033048,-0.008142,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306480,-0.298546,-0.306480,-0.306480,netscan,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684150,0,-0.004520,-0.002387,-0.124520,-0.431610,-0.292571,-0.454575,-0.289197,-0.118756,-0.119291,-0.119111,-0.753681,-0.481517,-0.478594,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.293435,-0.294249,-0.306564,-0.299233,-0.306564,-0.306564,camoverflow,1,0,0,1
684151,0,-0.004520,-0.002387,-0.084516,-0.431610,-0.292571,0.197830,-0.289197,-0.118756,-0.119291,-0.119111,1.148129,0.204570,0.274130,-0.173937,0.099664,-0.277044,-0.293967,-0.229925,-0.078668,-0.304146,-0.299700,-0.293435,-0.294249,-0.265643,0.039041,-0.265643,-0.265643,rudeadyet,1,0,0,1
684152,0,-0.004508,-0.002387,-0.004509,-0.431610,-0.290481,1.502640,3.110134,-0.069600,-0.069931,-0.069817,1.148129,0.033048,0.085949,1.901923,1.434781,2.650890,2.949074,2.338752,1.565251,-0.302097,-0.295678,-0.282011,-0.272200,-0.289664,-0.271293,-0.289664,-0.289664,normal,0,1,0,0
684153,0,-0.004520,-0.002387,-0.084516,-0.431610,-0.289681,-0.454575,-0.289197,-0.106902,-0.095484,-0.101279,3.049938,0.204570,1.026854,-0.354446,-0.321952,-0.277044,-0.293967,-0.335307,-0.299082,-0.304146,-0.299700,-0.290499,-0.282918,-0.303687,-0.287343,-0.303687,-0.303687,rudeadyet,1,0,0,1


---------------------------------------

**Create Model & Train Model**

In [5]:
df_train = df_train.drop('traffic', axis=1)
df_test = df_test.drop('traffic', axis=1)

In [6]:
x_columns = df_train.columns.drop('is_attack')

le = LabelEncoder()
le.fit(df_train["is_attack"].values)

x = df_train[x_columns].values
y = df_train["is_attack"].values
y = le.transform(y)

In [7]:
x_columns_val = df_test.columns.drop('is_attack')

le = LabelEncoder()
le.fit(df_test["is_attack"].values)

x_val = df_test[x_columns].values
y_val = df_test["is_attack"].values
y_val = le.transform(y_val)

In [8]:
# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

In [9]:
x_test.shape,y_test.shape

((513117, 31), (513117,))

In [10]:
x_train.shape,y_train.shape

((1539348, 31), (1539348,))

In [11]:
x_val.shape, y_val.shape

((684155, 31), (684155,))

In [12]:
import numpy as np

# Random state definition
random_state=42
np.random.seed(random_state)

In [13]:
from sklearn import metrics

# Classification Metrics: Accuracy, Recall, Precision & F1Score
def compute_metrics(y_test, pred):
    print('Accuracy:', metrics.accuracy_score(y_test, pred))
    print('Recall:', metrics.recall_score(y_test, pred, average = 'weighted'))
    print('Precision:', metrics.precision_score(y_test, pred, average = 'weighted'))
    print('F1-Score:', metrics.f1_score(y_test, pred, average = 'weighted'))

In [14]:
# Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

def extra_tree_classifier(x_train, y_train, x_columns):
    # Usage of ExtraTreesClassifier for feature selection
    extra_tree_forest = ExtraTreesClassifier(n_estimators=5, criterion='entropy', max_features=2, random_state=random_state)
    extra_tree_forest.fit(x_train, y_train)
    feature_importances = extra_tree_forest.feature_importances_
    feature_importance_normalized = np.std([tree.feature_importances_ for tree in  extra_tree_forest.estimators_], axis = 0)

    # Plot the feature importances
    plt.rcParams["figure.figsize"] = (70, 40)
    plt.bar(x_columns, feature_importance_normalized, align='center')
    plt.xlabel('Feature Index')
    plt.ylabel('Feature Importance')
    plt.title('Comparison of different feature importances in the current dataset')
    plt.xticks(rotation=90)
    plt.show()

In [15]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

def conf_matrix(y_test, prediction, array_dimension = 2):
    if (array_dimension == 2):
        predict_classes = np.argmax(prediction, axis = 1)
        expected_classes = np.argmax(y_test, axis = 1)
    elif (array_dimension == 1):
        predict_classes = prediction
        expected_classes = y_test
    
    cm = confusion_matrix(expected_classes, predict_classes)
    cmd = ConfusionMatrixDisplay(cm, display_labels=['Normal', 'Malicious'])

    # Plot size
    fig, ax = plt.subplots(figsize=(4,4))
    
    # Add axis labels and rotate them
    ax.set_xlabel('Predicted labels', rotation=0, labelpad=10, fontsize=11)
    ax.set_ylabel('True labels', rotation=90, labelpad=10, fontsize=11)
    
    cmd.plot(ax=ax, cmap=plt.cm.Blues, colorbar=False)

    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['Normal', 'Malicious'])
    ax.set_yticklabels(['Normal', 'Malicious'])
    
    ax.tick_params(axis='x', pad=10)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center', va='center')
    ax.tick_params(axis='y', pad=10)
    ax.set_yticklabels(ax.get_yticklabels(), rotation=90, ha='center', va='center')
    
    # Adjust colorbar size
    cax = plt.gcf().axes[-1]
    cax.tick_params(labelsize=11)  # Adjust the font size of colorbar labels
    cax = fig.add_axes([ax.get_position().x1+0.03,ax.get_position().y0,0.02,ax.get_position().height])
    plt.colorbar(cmd.im_, cax=cax)    

In [16]:
def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=False):
    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshowac(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=90)
        plt.yticks(tick_marks, target_names)
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [17]:
%%time

# Algorithm: KNEIGHBORS
from sklearn.neighbors import KNeighborsClassifier

neighbors = KNeighborsClassifier(n_neighbors=2, weights='uniform')
neighbors.fit(x_train, y_train)
neighbors_prediction = neighbors.predict(x_test)

print(f'[METRICS] Kneighbors Classifier')
compute_metrics(y_test, neighbors_prediction)

print(f'\n[CMATRIX] Kneighbors Confusion Matrix')
conf_matrix(y_test, neighbors_prediction, 1)

KeyboardInterrupt: 